In [1]:
from imblearn.over_sampling import SMOTE, RandomOverSampler

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
path = '../../data/Change-Level-Prediction-Data-20191107T052353Z-001/Change-Level-Prediction-Data/ICSE-2016-PROMISE DefectData/'
train_fname = 'camel-1.2.csv'
test_fname = 'camel-1.4.csv'

In [24]:
# Get the data
train_df = pd.read_csv(path + train_fname)
test_df = pd.read_csv(path + test_fname)

In [25]:
train_df.head()

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,camel,1.2,org.apache.camel.component.irc.IrcErrorLogger,9,0,0,5,24,0,1,...,1.0,1,0.000000,0.370370,0,0,22.555556,1,0.8889,0
1,camel,1.2,org.apache.camel.ProducerTemplate,27,1,0,6,27,351,1,...,0.0,0,0.000000,0.412037,0,0,0.000000,1,1.0000,3
2,camel,1.2,org.apache.camel.impl.JndiRegistry,9,1,0,3,18,30,1,...,1.0,0,0.000000,0.355556,0,0,8.000000,1,0.7778,0
3,camel,1.2,org.apache.camel.component.cxf.CxfSoapConsumer,7,1,0,27,50,7,3,...,1.0,3,0.000000,0.416667,0,0,26.428571,1,0.7143,0
4,camel,1.2,org.apache.camel.processor.TryProcessor,8,2,0,10,28,0,1,...,1.0,3,0.684211,0.371429,1,1,26.500000,2,1.0000,9


In [26]:
# Remove unnecessary columns
train_df.drop(['name', 'version', 'name.1'], inplace=True, axis=1)
test_df.drop(['name', 'version', 'name.1'], inplace=True, axis=1)

In [27]:
# Get the X and y for train and test sets
x_train = train_df.drop('bug', axis=1)
y_train = train_df['bug']

x_test = test_df.drop('bug', axis=1)
y_test = test_df['bug']

In [28]:
np.unique(y_train), np.unique(y_test)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 28]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 13, 17]))

In [19]:
# Purge the weird rows
idx = np.where(y_train < 2)[0]
x_train = np.array(x_train)[idx]
y_train = y_train[idx]

idx = np.where(y_test < 2)[0]
x_test = np.array(x_test)[idx]
y_test = y_test[idx]

In [8]:
# Alternate purge method
y_train[y_train > 1] = 1
y_test[y_test > 1] = 1

/Users/ryedida/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ryedida/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
# Purge method #3
y_train[y_train > 1] = 0
y_test[y_test > 1] = 0

/Users/ryedida/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ryedida/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((608, 20), (608,), (872, 20), (872,))

In [31]:
sampler = RandomOverSampler()

In [32]:
x_train, y_train = sampler.fit_resample(x_train, y_train)

In [33]:
clf = RandomForestClassifier()
clf.fit(x_train, y_train)

/Users/ryedida/Library/Python/3.7/lib/python/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [23]:
preds = clf.predict(x_test)
f1_score(y_test, preds)

0.1925133689839572

Having tried both the purging methods and the RandomForest classifier, neither are close. With the first method, it yields about 12% F1; with the other, it yields about 28%.

Let's try with a different classifier

In [53]:
from sklearn.naive_bayes import GaussianNB

In [54]:
nb = GaussianNB()
nb.fit(x_train, y_train)

preds = nb.predict(x_test)
f1_score(y_test, preds)

0.5541125541125541

What if we used the old purging method where we remove the rows?

In [65]:
nb = GaussianNB()
nb.fit(x_train, y_train)

preds = nb.predict(x_test)
f1_score(y_test, preds)

0.3687150837988827

So there seems to be a variance here, and I wonder if their results are based on the second purging method (setting them to 1).

What if we set them to 0 (method 3)?

In [77]:
nb = GaussianNB()
nb.fit(x_train, y_train)

preds = nb.predict(x_test)
f1_score(y_test, preds)

0.319634703196347